# 🔧 Fine-Tuning ResNet-50 on Real, Synthetic, and Mixed Fire Datasets

This notebook fine-tunes ResNet-50 classification models using three dataset compositions:
- 100% real images from the D-Fire dataset
- 100% synthetic images from the Yunnan UE5-generated dataset
- 50/50 mixed composition using the `FireClassificationMixedDataset` class

The goal is to assess whether fine-tuning improves model performance compared to feature extraction, especially when only limited real data is available. Fine-tuning is performed by unfreezing the final ResNet-50 block (`layer4`) and fully connected layer (`fc`), while keeping earlier layers frozen.

Each model is trained for 5 epochs using the same training loop defined in `train_model.py`, with validation on the held-out D-Fire validation set. Results will be compared directly to their frozen counterparts from Phase 1.


## 📦 Notebook Setup: Mount Drive & Clone GitHub Repo

This cell ensures the notebook is reproducible in any new Colab session by:

- Mounting your Google Drive (to access datasets, secrets, and checkpoints)
- Loading your GitHub token from Drive
- Cloning the fire-detection-dissertation repository
- Navigating into the correct folder
- Setting Git identity for future commits

⚠️ **Note:** This cell must be run every time you open this notebook in a new Colab session.


In [ ]:
# 🔧 Minimal Colab setup for any working notebook

# 1. Mount Google Drive
import os
from google.colab import drive
if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")

# 2. Load GitHub token securely from Drive
token_path = "/content/drive/MyDrive/fire-detection-dissertation/secrets/github_token.txt"
with open(token_path, "r") as f:
    token = f.read().strip()

# 3. Clone the GitHub repo (force fresh clone for safety)
username = "Misharasapu"
repo = "fire-detection-dissertation"
clone_url = f"https://{token}@github.com/{username}/{repo}.git"
repo_path = f"/content/{repo}"

# Optional: Remove old clone (safe to rerun)
!rm -rf {repo_path}

# Clone fresh and move into the repo
%cd /content
!git clone {clone_url}
%cd {repo}

# 4. Set Git identity (required in Colab sessions)
!git config --global user.name "Misharasapu"
!git config --global user.email "misharasapu@gmail.com"


## 🧩 Step 1: Define Dataset Type, Paths, and Model Filenames

In this step, we define the training configuration for each model to be fine-tuned. Three dataset types are used:

- 🔵 `real`: 100% real images (D-Fire)
- 🟠 `synthetic`: 100% synthetic images (Yunnan UE5)
- 🟣 `mixed`: 50% synthetic, 50% real

We also define model filenames dynamically and ensure reproducibility with a fixed random seed. The total number of training samples is kept constant for the mixed dataset (5,260), and filenames are structured to reflect the data composition used during training.


In [ ]:
import os
import random
import torch
import numpy as np

# 🔧 Adjustable configuration
syn_ratio = 0.50                    # Used only for the mixed dataset
total_samples = 5260                # Fixed total sample size for mixed dataset

# 🗂️ Paths to training data
real_image_dir = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/D-Fire/train/images"
real_label_dir = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/real/D-Fire/train/labels"

# Yunnan synthetic (confirmed 'synthetic_all' exists)
syn_image_dir  = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/synthetic/yunnan/synthetic_all/images"
syn_label_dir  = "/content/drive/MyDrive/fire-detection-dissertation/data/raw/synthetic/yunnan/synthetic_all/labels"

# 🧠 Model filenames based on dataset type (match existing Phase 2 naming)
model_paths = {
    "real":       "resnet_outdoor_real_100_ft_phase2.pt",
    "synthetic":  "resnet_outdoor_synthetic_100_ft_phase2.pt",
    "mixed":      f"resnet_outdoor_{int(syn_ratio*100)}syn_{int((1-syn_ratio)*100)}real_ft_phase2.pt"
}

# ✅ Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)


## 🧼 Step 2: Define Image Transformations

We define the image transformation pipeline used across all datasets. Images are resized to 224×224, normalized to ImageNet statistics, and converted to PyTorch tensors.

This ensures compatibility with the input format expected by ResNet-50, which was pretrained on ImageNet.


In [ ]:
# --- Step 2: Image transforms (train + val only, no augmentation, no Normalize) ---

from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),            # 👈 no Normalize
])

print("Transforms set: Resize(224,224) → ToTensor()  [no normalization]")

Transforms set: Resize(224,224) → ToTensor()  [no normalization]


## ⚙️ Step 3: Load Dataset, Pretrained Model, and Configure Fine-Tuning

Based on the selected training mode (`"real"`, `"synthetic"`, or `"mixed"`), we perform the following:

1. Load the appropriate dataset class with the predefined transform:
   - `FireClassificationDataset` for real images (D-Fire)
   - `FireClassificationSyntheticDataset` for synthetic images (Yunnan UE5)
   - `FireClassificationMixedDataset` for a fixed-ratio combination of both

2. Split the full dataset into training and validation subsets using an 80/20 split with a fixed random seed (`seed=42`) to ensure reproducibility.

3. Load the corresponding Phase 1 model (`*_100.pt`), which was trained using feature extraction (frozen base, trained `fc` layer only).

4. Construct the Adam optimizer using only the parameters marked with `requires_grad=True`. Fine-tuning will be handled automatically inside the `train_model()` helper by unfreezing `layer4` and `fc` when `fine_tune=True`.

This setup ensures consistent dataset usage and training conditions across all modes, and enables us to isolate the impact of fine-tuning deeper convolutional layers on classification performance.


In [ ]:
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torch import nn, optim
import os

from utils.fire_classification_dataset import (
    FireClassificationDataset,
    FireClassificationMixedDataset
)

# 🔘 SELECT WHICH MODEL TO TRAIN
selected_mode = "real"  # Options: "real", "synthetic", "mixed"
print(f"\n📦 Selected mode: {selected_mode.upper()}")

# 🖥️ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
using_gpu = torch.cuda.is_available()
print(f"🖥️ Using device: {device} ({'GPU enabled' if using_gpu else 'CPU only'})")

# ✅ Training parameters
batch_size = 32
num_epochs = 5
learning_rate = 1e-4
print(f"📌 Training config → Batch size: {batch_size}, Epochs: {num_epochs}, LR: {learning_rate}")

# ✅ Build dataset by mode
if selected_mode == "real":
    print("🔹 Loading 100% real dataset (D-Fire)...")
    full_dataset = FireClassificationDataset(
        image_dir=real_image_dir,
        label_dir=real_label_dir,
        transform=transform,
        dataset_type="real"
    )
    pretrained_path = "/content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_phase1.pt"

elif selected_mode == "synthetic":
    print("🔸 Loading 100% synthetic dataset (Yunnan)...")
    full_dataset = FireClassificationDataset(
        image_dir=syn_image_dir,
        label_dir=syn_label_dir,
        transform=transform,
        dataset_type="synthetic"
    )
    pretrained_path = "/content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_synthetic_100_phase1.pt"

elif selected_mode == "mixed":
    print("🟣 Loading mixed dataset (Real + Synthetic)...")
    print(f"   → Synthetic ratio: {syn_ratio:.2f}, Total samples: {total_samples}")
    full_dataset = FireClassificationMixedDataset(
        real_image_dir=real_image_dir,
        real_label_dir=real_label_dir,
        syn_image_dir=syn_image_dir,
        syn_label_dir=syn_label_dir,
        syn_ratio=syn_ratio,
        total_samples=total_samples,
        transform=transform
    )
    pretrained_path = (
        f"/content/drive/MyDrive/fire-detection-dissertation/models/"
        f"resnet_outdoor_{int(syn_ratio*100)}syn_{int((1-syn_ratio)*100)}real_phase1.pt"
    )
else:
    raise ValueError("❌ Invalid mode. Choose from: 'real', 'synthetic', or 'mixed'.")

# ✅ Split into train / val
train_ratio = 0.8
train_size = int(train_ratio * len(full_dataset))
val_size = len(full_dataset) - train_size
generator = torch.Generator().manual_seed(42)

train_data, val_data = random_split(full_dataset, [train_size, val_size], generator=generator)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
print(f"📊 Dataset split → Train: {train_size}, Validation: {val_size}")

# ✅ Load Phase‑1 checkpoint safely
print(f"\n🔧 Loading pretrained Phase 1 model: {pretrained_path}")
if not os.path.exists(pretrained_path):
    raise FileNotFoundError(f"Checkpoint not found at: {pretrained_path}")

# Note: torchvision warns about 'pretrained=True' in newer versions; this keeps parity with your earlier runs.
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)

state = torch.load(pretrained_path, map_location=device)
model.load_state_dict(state)
model = model.to(device)
print("✅ Model loaded and moved to device.")

# 🔓 FINE‑TUNE POLICY (Phase 2): unfreeze only layer4 + fc
print("\n🧩 Fine‑tuning setup → Freeze all, then unfreeze layer4 + fc")
for p in model.parameters():
    p.requires_grad = False
for p in model.layer4.parameters():
    p.requires_grad = True
for p in model.fc.parameters():
    p.requires_grad = True

# Show a quick summary of what's trainable
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"🔎 Trainable params: {trainable_params:,} / {total_params:,} "
      f"({100.0 * trainable_params / total_params:.2f}% of model)")

# ✅ Optimizer only on trainable params
optimizer = optim.Adam((p for p in model.parameters() if p.requires_grad), lr=learning_rate)
print("🛠️ Optimizer configured (Adam) on layer4 + fc only — fine‑tuning mode active.")

# ✅ Save path for the fine‑tuned model
save_path = f"/content/drive/MyDrive/fire-detection-dissertation/models/{model_paths[selected_mode]}"
print(f"💾 Fine‑tuned model will be saved to: {save_path}")



📦 Selected mode: REAL
🖥️ Using device: cpu (CPU only)
📌 Training config → Batch size: 32, Epochs: 5, LR: 0.0001
🔹 Loading 100% real dataset (D-Fire)...
📊 Dataset split → Train: 13777, Validation: 3445

🔧 Loading pretrained Phase 1 model: /content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_phase1.pt


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Model loaded and moved to device.

🧩 Fine‑tuning setup → Freeze all, then unfreeze layer4 + fc
🔎 Trainable params: 14,968,834 / 23,512,130 (63.66% of model)
🛠️ Optimizer configured (Adam) on layer4 + fc only — fine‑tuning mode active.
💾 Fine‑tuned model will be saved to: /content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_ft_phase2.pt


## 🔁 Step 4: Train the Fine-Tuned Model (layer4 + fc)

In this step, we begin training the selected model using the `train_model()` helper function.

- `fine_tune=True` ensures that both `layer4` and `fc` are unfrozen and trained.
- The model weights are initialized from the Phase 1 checkpoint (trained with feature extraction).
- The model will be trained for 5 additional epochs using the same dataset composition as in Phase 1.
- Results will be printed at the end of each epoch and the best-performing model (by F1 score) will be saved to the specified path.


In [7]:
from utils.train_model import train_model
from torch.nn import CrossEntropyLoss

# ✅ Define loss function
criterion = CrossEntropyLoss()

# ✅ Start training (with fine-tuning)
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device,
    save_path=save_path,
    print_every=1,
    print_batch_loss=False
)



🔍 Model device: cpu
📊 Trainable parameters: 14968834

🔁 Epoch 1/5


✅ Epoch [1/5] | Train Loss: 0.1366 | Val Loss: 0.0937 | Acc: 0.9689 | Precision: 0.9497 | Recall: 0.9343 | F1: 0.9419 | Time: 16591.3s
💾 New best model saved (F1: 0.9419) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_ft_phase2.pt

🔁 Epoch 2/5


✅ Epoch [2/5] | Train Loss: 0.0499 | Val Loss: 0.1230 | Acc: 0.9608 | Precision: 0.9553 | Recall: 0.8967 | F1: 0.9250 | Time: 4633.5s

🔁 Epoch 3/5


✅ Epoch [3/5] | Train Loss: 0.0321 | Val Loss: 0.1272 | Acc: 0.9704 | Precision: 0.9366 | Recall: 0.9548 | F1: 0.9456 | Time: 4691.5s
💾 New best model saved (F1: 0.9456) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_ft_phase2.pt

🔁 Epoch 4/5


✅ Epoch [4/5] | Train Loss: 0.0221 | Val Loss: 0.1066 | Acc: 0.9724 | Precision: 0.9484 | Recall: 0.9494 | F1: 0.9489 | Time: 4680.4s
💾 New best model saved (F1: 0.9489) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_ft_phase2.pt

🔁 Epoch 5/5


✅ Epoch [5/5] | Train Loss: 0.0145 | Val Loss: 0.1251 | Acc: 0.9733 | Precision: 0.9614 | Recall: 0.9386 | F1: 0.9499 | Time: 4607.4s
💾 New best model saved (F1: 0.9499) → /content/drive/MyDrive/fire-detection-dissertation/models/resnet_outdoor_real_100_ft_phase2.pt


In [1]:
# ✅ 1. Navigate to the Git-tracked repo folder
%cd /content/fire-detection-dissertation

# ✅ 2. Move the notebook from Drive into the repo so Git can track it
!cp /content/drive/MyDrive/fire-detection-dissertation/notebooks/06_train_resnet_finetuned.ipynb /content/fire-detection-dissertation/notebooks/

# Optional: confirm it's now inside the repo
!ls notebooks/

# ✅ 3. Stage the notebook for commit
!git add notebooks/06_train_resnet_finetuned.ipynb

# ✅ 4. Commit with a message
!git commit -m "Refactored Phase 2 fine-tuning notebook with unnormalised pipeline and updated dataset handling"

# ✅ 5. Push to GitHub
!git push


[Errno 2] No such file or directory: '/content/fire-detection-dissertation'
/content
cp: cannot stat '/content/drive/MyDrive/fire-detection-dissertation/notebooks/06_train_resnet_finetuned.ipynb': No such file or directory
ls: cannot access 'notebooks/': No such file or directory
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
